In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

: 

In [ ]:
dataset=pd.read_csv("../raw_data/dataset.csv")

In [ ]:
dataset.head()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset[dataset.duplicated()]

In [ ]:
dataset.info()

In [ ]:
dataset.drop(columns=['RowNumber', 'CustomerId', 'Surname'],inplace=True)

In [ ]:
sns.countplot(x="Exited",data=dataset)

In [ ]:
num_cols=dataset.select_dtypes(np.number).columns
num_cols

In [ ]:
dataset[num_cols].hist(figsize=(12,8))

In [ ]:
for col in num_cols:
    sns.boxplot(x=dataset[col])
    plt.title(col)
    plt.show()

In [ ]:
cat=dataset.select_dtypes(pd.Categorical).columns
cat

In [ ]:
for col in cat:
    print(dataset[col].value_counts())
    sns.countplot(x=col, data=dataset)

In [ ]:
for i in dataset:
  print(f"{i}=={dataset[i].unique()}")

In [ ]:
dataset["Gender"]=dataset["Gender"].map({"Male":1,"Female":0})

In [ ]:
df=pd.get_dummies(data=dataset,columns=["Geography"],dtype=int,drop_first=True)

In [ ]:
df

In [ ]:
features=['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts','EstimatedSalary','Gender',
       'HasCrCard', 'IsActiveMember','Geography_Germany', 'Geography_Spain']
x=df[features]
y=df['Exited']


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=12)

In [ ]:
x_test

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_features=['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts','EstimatedSalary']
scaler=StandardScaler()
x_train[scaled_features]=scaler.fit_transform(x_train[scaled_features])
x_test[scaled_features]=scaler.transform(x_test[scaled_features])

In [ ]:
x_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random=RandomForestClassifier(n_estimators=300,random_state=12)

In [ ]:
from imblearn.over_sampling import SMOTE
sm=SMOTE()
x_res, y_res = sm.fit_resample(x_train, y_train)


In [ ]:
random.fit(x_res,y_res)

In [ ]:
y_pred=random.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,recall_score,f1_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))
print("recall_score:",recall_score(y_test,y_pred))
print("f1_score:",f1_score(y_test,y_pred))

In [ ]:
importances=random.feature_importances_
indices=np.argsort(importances)[::-1]
feature_name=[features[i] for i in indices]

In [ ]:
sns.barplot(y=feature_name,x=importances[indices])

In [ ]:
import joblib
joblib.dump(random,'../ml_model/model.pkl')
joblib.dump(scaler,'../ml_model/scaler.pkl')